## 인터파크티켓 크롤링
#### 공연 개별 상세페이지에서 정보 가져오기

In [126]:
from selenium import webdriver
import datetime
import time

path="C:/Users/YYJ/Desktop/Programming/Python/Crawling/chromedriver.exe"
driver=webdriver.Chrome(path)

In [147]:
def get_data(p_type):
    
    p_info={'Type':'', 'Title':'', 'TicketOpen':'', 'Venue':'', 'StartDate':'', 'EndDate':'', 'Performer':'', 'Time':'',
           'RunningTime':'', 'Price':'', 'BuyTicket':'인터파크', 'PosterSrc':'', 'RegisteredTime':''}
    performer=[]
    
#     #RegisteredTime
#     p_info['RegisteredTime']=datetime.datetime.now()
    
    # Type
    p_info['Type']=p_type
    
    # Title
    p_info['Title']=driver.find_element_by_id('IDGoodsName').text
    
    elem=driver.find_element_by_id('TabA')
    
    #PosterSrc
    p_info['PosterSrc']=elem.find_element_by_tag_name('img').get_attribute('src')
    
    # Price
    try:
        for tr in elem.find_elements_by_tag_name('tr'):
            td=tr.find_elements_by_tag_name('td')
            p_info['Price']=p_info['Price']+td[0].text+' '+td[2].text+', '
        p_info['Price']=p_info['Price'][:-2]
    except IndexError:
        pass
    
    # Venue, StartDate, EndDate, Performer
    for dl in elem.find_elements_by_tag_name('dl'):
        info_type=dl.find_element_by_tag_name('dt')
        info_data=dl.find_element_by_tag_name('dd')
        
        if info_type.text=='장소':
            p_info['Venue']=info_data.text
        elif info_type.text=='기간':
            if '~' in info_data.text:
                p_info['StartDate']=info_data.text.split(' ', 3)[0]       # split('기준', '횟수')
                tmp=info_data.text.split(' ', 3)[2]
                if tmp!='오픈런':
                    p_info['EndDate']=tmp
            else:
                p_info['StartDate']=p_info['EndDate']=info_data.text.split(' ', 2)[0]
        elif info_type.text=='출연':
            driver.find_element_by_id('aTabC').click()
            driver.switch_to.frame('ifrTabC')
            for i in driver.find_elements_by_class_name('Actor'):
                performer.append(i.find_element_by_tag_name('h3').text)
            p_info['Performer']=performer
        else:
            pass
        
    driver.switch_to.default_content()

    # Time, RunningTime
    elem=driver.find_element_by_id('divTabArea')
    for data in elem.find_elements_by_class_name('Data'):
        info_type=data.find_element_by_tag_name('h4')
        info_data=data.find_element_by_class_name('Data_infoarea')
        
        if info_type.text=='공연시간 정보':
            p_info['Time']=info_data.text
        elif info_type.text=='상품관련정보':
            for tr in info_data.find_element_by_tag_name('table').find_elements_by_tag_name('tr'):
                for th in tr.find_elements_by_tag_name('th'):
                    if th.text=='공연시간':
                        p_info['RunningTime']=tr.find_element_by_tag_name('td').text.split('분', 2)[0]
                    else:
                        pass
        else:
            pass
    
    print(p_info)
    
    return p_info

In [148]:
def get_link():
    
    link=[]
    p_table=driver.find_element_by_xpath('/html/body/table/tbody/tr[2]/td[3]/div/div/div[2]/div/table/tbody')

    for row in p_table.find_elements_by_tag_name('tr'):
        link.append(row.find_element_by_tag_name('a').get_attribute('href'))
        
    return link

In [161]:
# p_type={'Mus':'1', 'Liv':'3', 'Dra':'2', 'Cla':'4', 'Eve&SubCa=Eve_T':'5', 'Fam':'6'}
p_type={'Dra':'2'}
p_data=[]       # 공연정보 리스트

for key in p_type:
    
    # [장르 - 전체보기 - 상품명순] 페이지 접근
    driver.get("http://ticket.interpark.com/TPGoodsList.asp?Ca="+key+"&Sort=4")
    time.sleep(0.5)
    
    # 각 공연의 상세페이지 주소 p_link에 가져오기
    p_link=get_link()
    
    # p_link의 주소로 상세페이지에 접근하여 데이터 가져오기
    for link in p_link:
        
        driver.get(link)
        time.sleep(0.5)
            
        # 상세페이지에서 얻은 공연정보 데이터를 p_data에 추가하기
        p_data.append(get_data(p_type[key]))
        

{'Type': '2', 'Title': '가족음악극 〈한여름 밤의 꿈〉', 'TicketOpen': '', 'Venue': '세종문화회관 M씨어터', 'StartDate': '2021.01.08', 'EndDate': '2021.01.31', 'Performer': '', 'Time': '화-목 11시 / 금 19시 30분 / 토 14시, 17시 / 일 14시\n(월 공연없음)', 'RunningTime': '70', 'Price': 'R석 40,000 원, S석 30,000 원, A석 20,000 원', 'BuyTicket': '인터파크', 'PosterSrc': 'http://ticketimage.interpark.com/Play/image/large/P0/P0002335_p.gif', 'RegisteredTime': ''}
{'Type': '2', 'Title': '갈매기 - 제주', 'TicketOpen': '', 'Venue': '예술공간 오이', 'StartDate': '2020.11.14', 'EndDate': '2020.11.29', 'Performer': '', 'Time': '매주 토,일 / 3시, 7시', 'RunningTime': '90', 'Price': '전석(예매가) 11,500 원', 'BuyTicket': '인터파크', 'PosterSrc': 'http://ticketimage.interpark.com/Play/image/large/20/20008225_p.gif', 'RegisteredTime': ''}
{'Type': '2', 'Title': '강제 결혼', 'TicketOpen': '', 'Venue': '대학로 코델 아트홀', 'StartDate': '2020.07.06', 'EndDate': '', 'Performer': '', 'Time': '매주 월요일 7시30분', 'RunningTime': '60', 'Price': '전석 30,000 원', 'BuyTicket': '인터파크', 'PosterSrc': 'http

{'Type': '2', 'Title': '그때 우리 - 부산', 'TicketOpen': '', 'Venue': '초콜릿팩토리', 'StartDate': '2020.11.06', 'EndDate': '2020.11.15', 'Performer': '', 'Time': '- 평일 오후 8시 / 토요일 오후 4시 / 일요일 오후 5시\n*월요일 공연없음', 'RunningTime': '85', 'Price': '', 'BuyTicket': '인터파크', 'PosterSrc': 'http://ticketimage.interpark.com/Play/image/large/20/20009333_p.gif', 'RegisteredTime': ''}
{'Type': '2', 'Title': '극한연애 - 부산', 'TicketOpen': '', 'Venue': '부산 초콜릿팩토리', 'StartDate': '2020.11.20', 'EndDate': '2021.01.03', 'Performer': '', 'Time': '［공연시간］\n평일: 오후 8시\n토요일: 오후 4시, 7시\n일요일: 오후 5시\n*월, 화, 수 공연없음\n\n*단 12월21일(월)~31일(목) 2주간\n매일 공연 있음 (월, 화, 수, 목, 금, 토, 일)\n\n［크리스마스 주 공연시간］\n*12월24일(목): 오후 3시, 5시 30분, 7시 30분\n*12월25일(금): 오전 11시, 오후 1시, 4시, 7시\n*12월26일(토): 오후 1시, 4시\n*12월27일(일): 오후 2시, 5시\n*1월1일(금): 오후 2시, 5시', 'RunningTime': '90', 'Price': '전석(평일 1인관람권) 12,900 원', 'BuyTicket': '인터파크', 'PosterSrc': 'http://ticketimage.interpark.com/Play/image/large/20/20009495_p.gif', 'RegisteredTime': ''}
{'Type': '2', 'Title': '끝까

{'Type': '2', 'Title': '배리어프리 뮤지컬', 'TicketOpen': '', 'Venue': '세종문화회관 M씨어터', 'StartDate': '2020.11.11', 'EndDate': '2020.11.15', 'Performer': '', 'Time': '평일 20:00 / 토 14:00, 19:00 / 일 14:00', 'RunningTime': '150', 'Price': 'R석 70,000 원, S석 50,000 원', 'BuyTicket': '인터파크', 'PosterSrc': 'http://ticketimage.interpark.com/Play/image/large/P0/P0002328_p.gif', 'RegisteredTime': ''}
{'Type': '2', 'Title': '뷰티풀라이프', 'TicketOpen': '', 'Venue': 'JTN 아트홀 4관', 'StartDate': '2018.02.20', 'EndDate': '', 'Performer': ['조준', '김원진', '김진호', '김수현', '김희정', '김가현', '정경식', '권순미'], 'Time': '금: 20시 / 토: 15시, 18시 / 일: 14시\n\n☞ 뷰티풀라이프 공식 블로그 바로가기 : https://blog.naver.com/ziancompany\n☞ 뷰티풀라이프 페이스북 바로가기 : https://www.facebook.com/ziancompany/\n☞ 뷰티풀라이프 인스타그램 바로가기 : https://www.instagram.com/ziancompany\n☞ 뷰티풀라이프 트위터 바로가기 : twitter.com/zipcompany', 'RunningTime': '90', 'Price': '일반석 40,000 원', 'BuyTicket': '인터파크', 'PosterSrc': 'http://ticketimage.interpark.com/Play/image/large/18/18001569_p.gif', 'RegisteredTime'

KeyboardInterrupt: 

In [162]:
## MySql DB에 데이터 넣기

import pymysql

conn=pymysql.connect(host='localhost', user='root', password='asdf1234', db='grad_test', charset='utf8mb4')
try:
    with conn.cursor() as cursor:
        sql='INSERT ignore INTO shows (type, title, venue, start_date, end_date, time, ticket_open, price, running_time, buy_ticket, poster_src, registered_time) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'
        for r in p_data:
            cursor.execute(sql, (r['Type'], r['Title'], r['Venue'], r['StartDate'], r['EndDate'], r['Time'], r['TicketOpen'], r['Price'], r['RunningTime'], r['BuyTicket'], r['PosterSrc'], datetime.datetime.now()))
            conn.commit()
finally:
    conn.close()

C:\ProgramData\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1265, "Data truncated for column 'end_date' at row 1")
  result = self._query(query)
C:\ProgramData\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1366, "Incorrect integer value: '상품상세정보 확인' for column 'running_time' at row 1")
  result = self._query(query)
